Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
# import os, sys
# in_colab = 'google.colab' in sys.modules

# # If you're in Colab...
# if in_colab:
#     # Pull files from Github repo
#     os.chdir('/content')
#     !git init .
#     !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
#     !git pull origin master
    
#     # Install required python packages
#     !pip install -r requirements.txt
    
#     # Change into directory for module
#     os.chdir('module3')

In [0]:
data = 'C:\\Users\\vanda\\Desktop\\'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(data+'\\data\\tanzania\\train_features.csv'), 
                 pd.read_csv(data+'\\data\\tanzania\\train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(data+'\\data\\tanzania\\test_features.csv')
sample_submission = pd.read_csv(data+'\\data\\tanzania\\sample_submission.csv')

In [102]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(train)
profile

Number of variables,41
Number of observations,59400
Total Missing (%),1.9%
Total size in memory,19.0 MiB
Average record size in memory,336.0 B
Numeric,10
Categorical,30
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [0]:
import numpy as np
def data_clean(df):
    #here I am going to try to clean the data before splitting it, so I can make a full map plot
    
    #prevent copy warning
    df = df.copy()
    
    #replace very small value with zero
    df['latitude'] = df['latitude'].replace(-2e-08, 0)
    
    #Now replace all zeros on lat and lon with nans
    rep_xzero = ['latitude', 'longitude', 'construction_year', 'population']
    for col in rep_xzero:
        df[col] = df[col].replace(0, np.nan)
        
    
    #change date recorded to datetime
    #df['date_recorded'] = pd.to_datetime(df['date_recorded'], infer_datetime_format=True)
    
    #creating years since construction feature
    #df['years_since_construction'] = df['date_recorded'].dt.year - df['construction_year']
    
    #drop duplicated column
    to_drop = ['quantity_group', 'extraction_type_group', 'waterpoint_type_group', 'num_private', 'scheme_name']
    df = df.drop(columns=to_drop)
    
    return df

In [0]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, train_size=0.8, stratify=train['status_group'], random_state=666)

In [0]:
#clean data through cleaning function

train = data_clean(train)
val = data_clean(val)
test = data_clean(test)

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

In [0]:
#externaly encoded
cardinal_num = 35

y_target = 'status_group'

train_feats = train.drop(columns=[y_target, 'id'])

num_feats = list(train_feats.select_dtypes('number').columns)

cardinality = train_feats.select_dtypes(exclude='number').nunique()

cat_feats = list(cardinality[cardinality <= cardinal_num].index)

ord_feats = list(cardinality[cardinality > cardinal_num].index)

features = num_feats + cat_feats

X_train = train[features]
X_train_ord = train[ord_feats]
y_train = train[y_target]

X_val = val[features]
X_val_ord = val[ord_feats]
y_val = val[y_target]

X_test = test[features]
X_test_ord = test[ord_feats]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_hot = encoder.fit_transform(X_train)
X_val_hot = encoder.transform(X_val)
X_test_hot = encoder.transform(X_test)

ordinal = ce.OrdinalEncoder()
X_train_ord = ordinal.fit_transform(X_train_ord)
X_val_ord = ordinal.transform(X_val_ord)
X_test_ord = ordinal.transform(X_test_ord)

X_train = pd.concat([X_train_hot, X_train_ord], axis=1)
X_val = pd.concat([X_val_hot, X_val_ord], axis=1)
X_test = pd.concat([X_test_hot, X_test_ord], axis=1)

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [0]:
# y_train = y_train.replace({'functional': 1, 'non functional': 0, 'functional needs repair':2})

In [168]:
#Randomized Search
pipeline = make_pipeline(
    #ce.OneHotEncoder(use_cat_names=True),
    #ce.TargetEncoder(),
    SimpleImputer(),
    RandomForestClassifier()
)

param_distro = {
    #'targetencoder__min_samples_leaf': randint(1,1000),
    #'targetencoder__smoothing': uniform(1,1000),
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(10,1000),
    'randomforestclassifier__criterion': ['gini', 'entropy'],
    'randomforestclassifier__max_depth': [5,10,15,20,25,30,35,40,45,50,55,60,67,70,75,80,85,90,95,100,None],
    'randomforestclassifier__min_samples_split': randint(2,200),
    'randomforestclassifier__min_samples_leaf': randint(1,100)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distro,
    n_iter=10,
    cv=3,
    verbose=10,
    return_train_score=True, 
    random_state=666,
    n_jobs=-1
)

search.fit(X_train, list(y_train))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done  11 out of  30 | elapsed:   49.3s remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:  1.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  1.1min remaining:   38.1s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  1.2min remaining:   21.3s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  1.7min remaining:   11.4s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.7min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
   ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'simpleimputer__strategy': ['mean', 'median'], 'randomforestclassifier__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A785073160>, 'randomforestclassifier__criterion': ['gini', 'entropy'], 'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, 3...fier__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A785073400>},
          pre

In [150]:
print('Best Hyperparameters: ', search.best_params_)
print('Cross-validation MAE: ', -search.best_score_)

Best Hyperparameters:  {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 70, 'randomforestclassifier__min_samples_leaf': 37, 'randomforestclassifier__min_samples_split': 28, 'randomforestclassifier__n_estimators': 271, 'simpleimputer__strategy': 'mean'}
Cross-validation MAE:  -0.7622474747474748


In [176]:
pipeline = make_pipeline(
    #ce.OneHotEncoder(use_cat_names=True),
    #ce.TargetEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(criterion='gini', max_depth=70, min_samples_leaf=37, min_samples_split=28, n_estimators=271)
)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=70, max_features='auto', max_leaf_nodes=None,
     ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [177]:
print('Training Accuracy: ', pipeline.score(X_train, y_train))
print('Validation Accuracy: ', pipeline.score(X_val, y_val))

Training Accuracy:  0.7772516835016835
Validation Accuracy:  0.7648148148148148


In [0]:
imputer = SimpleImputer(strategy ='mean')
X_train_impute = imputer.fit_transform(X_train)
X_val_impute = imputer.transform(X_val)
X_test_impute = imputer.transform(X_test)

In [162]:
model = RandomForestClassifier(criterion='gini', max_depth=70, min_samples_leaf=37, min_samples_split=28, n_estimators=271)

model.fit(X_train_impute, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=70, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=37, min_samples_split=28,
            min_weight_fraction_leaf=0.0, n_estimators=271, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [164]:
print('Training Accuracy: ', model.score(X_train_impute, y_train))
print('Validation Accuracy: ', model.score(X_val_impute, y_val))

Training Accuracy:  0.7823653198653199
Validation Accuracy:  0.0


In [0]:
# ##Reload Data
# # Merge train_features.csv & train_labels.csv
# train = pd.merge(pd.read_csv(data+'\\data\\tanzania\\train_features.csv'), 
#                  pd.read_csv(data+'\\data\\tanzania\\train_labels.csv'))

# # Read test_features.csv & sample_submission.csv
# test = pd.read_csv(data+'\\data\\tanzania\\test_features.csv')
# sample_submission = pd.read_csv(data+'\\data\\tanzania\\sample_submission.csv')
# train, val = train_test_split(train, test_size=0.2, train_size=0.8, stratify=train['status_group'], random_state=666)

# train = data_clean(train)
# val = data_clean(val)
# test = data_clean(test)

In [0]:
# #for target encoder
# cardinality_size = 50

# y_target = 'status_group'

# train_features2 = train.drop(columns=[y_target, 'id'])

# num_features2 = list(train_features2.select_dtypes('number').columns)

# cardinalit = train_features2.select_dtypes(exclude='number').nunique()

# catigorical_features2 = list(cardinalit[cardinalit <= cardinality_size].index)

# features2 = num_features2 + catigorical_features2

# X_train2 = train[features]
# y_train2 = train[y_target]

# X_val2 = val[features2]
# y_val2 = val[y_target]

# X_test2 = test[features2]



In [0]:
# pipeline2 = make_pipeline(
#     ce.OneHotEncoder(use_cat_names=True),
#     #ce.TargetEncoder(),
#     SimpleImputer(),
#     RandomForestClassifier()
# )

# param_distro2 = {
#     #'targetencoder__min_samples_leaf': randint(1,1000),
#     #'targetencoder__smoothing': randint(1,1000),
#     'simpleimputer__strategy': ['mean', 'median'],
#     'randomforestclassifier__n_estimators': randint(10,1000),
#     'randomforestclassifier__criterion': ['gini', 'entropy'],
#     'randomforestclassifier__max_depth': [5,10,15,20,25,30,35,40,45,50,55,60,67,70,75,80,85,90,95,100,None],
#     'randomforestclassifier__min_samples_split': randint(2,200),
#     'randomforestclassifier__min_samples_leaf': randint(1,100)
# }

# search2 = RandomizedSearchCV(
#     pipeline2,
#     param_distributions=param_distro2,
#     n_iter=10,
#     cv=3,
#     verbose=10,
#     return_train_score=True, 
#     random_state=666,
#     n_jobs=-1
# )

# search2.fit(X_train2, list(y_train))

In [0]:

# print('Best Hyperparameters: ', search2.best_params_)
# print('Cross-validation MAE: ', -search2.best_score_)

In [0]:
# from sklearn.model_selection import GridSearchCV
# ###One more time to try and get the target encoder working

# ##Reload Data
# # Merge train_features.csv & train_labels.csv
# train = pd.merge(pd.read_csv(data+'\\data\\tanzania\\train_features.csv'), 
#                  pd.read_csv(data+'\\data\\tanzania\\train_labels.csv'))

# # Read test_features.csv & sample_submission.csv
# test = pd.read_csv(data+'\\data\\tanzania\\test_features.csv')
# sample_submission = pd.read_csv(data+'\\data\\tanzania\\sample_submission.csv')
# train, val = train_test_split(train, test_size=0.2, train_size=0.8, stratify=train['status_group'], random_state=666)

# train = data_clean(train)
# val = data_clean(val)
# test = data_clean(test)

# #for target encoder
# cardinality_size = 50

# y_target = 'status_group'

# train_features2 = train.drop(columns=[y_target, 'id'])

# num_features2 = list(train_features2.select_dtypes('number').columns)

# cardinalit = train_features2.select_dtypes(exclude='number').nunique()

# catigorical_features2 = list(cardinalit[cardinalit <= cardinality_size].index)

# features2 = num_features2 + catigorical_features2

# X_train2 = train[features]
# y_train2 = train[y_target]

# X_val2 = val[features2]
# y_val2 = val[y_target]

# X_test2 = test[features2]
# y_train2 = y_train2.replace({'functional': 1, 'non functional': 0, 'functional needs repair':2})

# pipeline3 = make_pipeline(
#     ce.TargetEncoder(),
#     SimpleImputer(),
#     RandomForestClassifier()
# )

# param_distro3 = {
#     'targetencoder__min_samples_leaf': randint(1, 1000), 
#     'targetencoder__smoothing': randint(1, 1000),
#     'simpleimputer__strategy': ['mean', 'median'],
#     'randomforestclassifier__n_estimators': randint(10,1000),
#     'randomforestclassifier__criterion': ['gini', 'entropy'],
#     'randomforestclassifier__max_depth': [5,10,15,20,25,30,35,40,45,50,55,60,67,70,75,80,85,90,95,100,None],
#     'randomforestclassifier__min_samples_split': randint(2,200),
#     'randomforestclassifier__min_samples_leaf': randint(1,100)
# }

# search3 = RandomizedSearchCV(
#     pipeline3,
#     param_distributions=param_distro3,
#     n_iter=10,
#     cv=3,
#     verbose=20,
#     return_train_score=True, 
#     random_state=666,
#     n_jobs=-1
# )

# search3.fit(X_train2, list(y_train2))

In [0]:
# print('Best Hyperparameters: ', search3.best_params_)
# print('Cross-validation MAE: ', -search3.best_score_)